In [ ]:
import matplotlib
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from pylab import rcParams
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, median_absolute_error
from sklearn.model_selection import cross_val_score

In [ ]:
src_path = 'indexProcessed.csv'
test_rate = 0.2
# test_num = 10                 # test number 

In [ ]:
df = pd.read_csv(src_path, sep = ",")
# df = df.drop(df.columns[[0]], axis = 1)
display(df)

In [ ]:
# Convert Date column to datetime
df.loc[:, 'Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')

# Change all column headings to be lower case, and remove spacing
df.columns = [str(x).lower().replace(' ', '_') for x in df.columns]

# Get month of each sample
# df['month'] = df['date'].dt.month

# # Sort by datetime
# df.sort_values(by='date', inplace=True, ascending=True)

df.head(10)

In [ ]:
index, times = print(df['index'][0]), 0
change_index = []
print(type(df))
for i in df['index']:
    if i != index:
        print(i, times)
        change_index.append(times)
        index = i
    times += 1
print(change_index, len(change_index)) ## find index to split stocks
# print(df[0: 2])
# print(df.iloc[0: 2])

In [ ]:
df.head(8495) # make sure the index is correct to split
# print(df[0: 2])
# print(df.iloc[0: 2])
# print(df.loc[10000, 'index'])
# df.loc[df['index'] == 'HSI'] # copy dataframe with case
# print(df.iloc[len(df) - 1: len(df)]) # find last index
# print(df.adj_close.shift(-2))

In [ ]:
# ax = matplotlib.pyplot.subplots(4, 4)
rcParams['figure.figsize'] = 6, 3

index, times = 0, 0

# while times < len(change_index):
#         if times == 0:
#                 ax = df.iloc[: change_index[times + 1]].plot(x = 'date', y = 'adj_close', style = 'b-', grid = True)
#                 index = change_index[times]
#                 ax.set_xlabel(df['index'][change_index[times]])
#                 times += 1
#                 continue

#         elif times == len(change_index) - 1:
#                 ax = df.iloc[change_index[times]:].plot(x = 'date', y = 'adj_close', style = 'b-', grid = True)
#                 ax.set_xlabel(df['index'][change_index[times]])
#                 break

#         else:
#                 ax = df.iloc[index: change_index[times + 1]].plot(x = 'date', y = 'adj_close', style = 'b-', grid = True)
#                 index = change_index[times]
#                 ax.set_xlabel(df['index'][change_index[times]])
#                 times += 1


## data process function

In [ ]:
def wash_data(X, test_rate):
    num_test = int(test_rate * len(X))
    num_train = len(X) - num_test

    X_train, Y_train = X[:num_train], X[:num_train].adj_close
    X_test, Y_test = X[num_train:], X[num_train:].adj_close

    date_val = X_test['date'].shift(1) ## x value to draw

    X_train = X_train[['high', 'low', 'close', 'volume']]
    X_test = X_test[['high', 'low', 'close', 'volume']]

    Y_train.shift(-1).fillna(Y_train.mean()) # make data to predict next day price

    X_train = X_train.shift(1).fillna(X_train.mean())

    X_test = X_test.shift(1).fillna(X_test.mean())

    Y_test = Y_test.shift(-1).fillna(Y_test.mean())

    return X_train, Y_train, X_test, Y_test, date_val

## Regression Switch Fuction (linear, polynomial and Losso)

In [ ]:
def regression_mode(X, test_rate, mode):

    # print(X)
    X_train, Y_train, X_test, Y_test, date_val = wash_data(X, test_rate)

    if mode == 'linear':
        regression = linear_model.LinearRegression()
        regression.fit(X_train, Y_train)
        print(pd.DataFrame(regression.coef_.T, X_train.columns, columns = ['Coefficient']))
        print(regression.intercept_)
        Y_pred = regression.predict(X_test)
        print("Coefficients: \n", regression.coef_)

    elif mode == 'polynomial':
        regression = PolynomialFeatures(degree = 2)
        x_poly = regression.fit_transform(X_train)
        model = linear_model.LinearRegression()
        model.fit(x_poly, Y_train)
        Y_pred = model.predict(regression.fit_transform(X_test))

    else: 
        # Lasso
        # from sklearn.linear_model import Lasso, LassoCV
        regression = linear_model.Lasso(alpha = 5)
        regression.fit(X_train, Y_train)
        print(pd.DataFrame(regression.coef_.T, X_train.columns, columns = ['Coefficient']))
        print(regression.intercept_)
        Y_pred = regression.predict(X_test)
        print("Coefficients: \n", regression.coef_)
            

    # The mean squared error
    print("Mean squared error: %.2f" % mean_squared_error(Y_test, Y_pred))
    # The coefficient of determination: 1 is perfect prediction
    print("Coefficient of determination: %.2f" % r2_score(Y_test, Y_pred))
    print("explained_variance_score: %.2f" % explained_variance_score(Y_test, Y_pred))
    mae = median_absolute_error(Y_test, Y_pred)
    print("Mae: %.2f" % mae)

   
    scores = cross_val_score(regression, X_test, Y_test, cv = 5)
    print(scores)

    rcParams['figure.figsize'] = 10,4
    ax = matplotlib.pyplot.subplots(1, 1)
    ax = plt.scatter(date_val, Y_test, color="blue", marker = '.')
    bx = plt.plot(date_val, Y_pred, color="red")

    # ax = plt.scatter(date_val[1: ], Y_test[: len(Y_test) - 1], color="blue", marker = '.')
    # bx = plt.plot(date_val[1: ], Y_pred[1: ], color="red")

## Main function

In [ ]:
index, times = 0, 0
model = 'linear' # polynomial or lasso

while times < len(change_index):
        print(df['index'][change_index[times]])
        if times == 0:
                regression_mode(df[:change_index[times + 1]], test_rate, model)
                index = change_index[times]
                times += 1
                continue

        elif times == len(change_index) - 1:
                regression_mode(df[change_index[times]:], test_rate, model)
                break

        else:
                regression_mode(df[index: change_index[times + 1]], test_rate, model)
                index = change_index[times]
                times += 1